In [20]:
import numpy as np
import json 
import pandas as pd 
from sklearn.model_selection import train_test_split 
import xgboost as xgb 

In [3]:
data = pd.read_csv("../data/kc_house_data.csv")

In [5]:
data.select_dtypes(include=["float64"]).columns

Index(['price', 'bathrooms', 'floors', 'lat', 'long'], dtype='object')

In [16]:
Y = data["price"]
X = data[["floors", "lat", "long"]]

x_train, x_test, y_train, y_test = train_test_split(X, Y , test_size=0.3)

xgb_reg = xgb.XGBRegressor(random_state=42, n_estimators=1000, learning_rate=0.05)
xgb_reg.fit(x_train, y_train, eval_set=[(x_test, y_test)], verbose=False)

pickle_file = "artifacts/xgb_model.pkl"
import pickle

with open(pickle_file, "wb") as f:
    pickle.dump(xgb_reg, f)

In [21]:
type(x_train)

pandas.core.frame.DataFrame

In [27]:
with open("artifacts/xgb_model.pkl", 'rb') as model_file:
    model = pickle.load(model_file)

request_body = '{"floors": 2.0, "lat": 47.5112, "long": -122.257}'


input_data = json.loads(request_body)
features = np.array([input_data.get('floors', 0), input_data.get('lat', 0), input_data.get('long', 0)]).reshape(1, -1)
prediction = model.predict(features)


prediction

array([563297.5], dtype=float32)

In [29]:
int(prediction[0])

563297

 Step 1

In [17]:
import boto3
import os
import subprocess

# Define your file paths
tar_file_path = 'artifacts/xgb_model.tar.gz'
pickle_file_path = 'artifacts/xgb_model.pkl'

# Step 1: Delete the existing tar file if it exists
if os.path.exists(tar_file_path):
    os.remove(tar_file_path)
    print(f'Deleted existing file: {tar_file_path}')

# Step 2: Ensure the pickle file exists at the expected location
if not os.path.exists(pickle_file_path):
    raise FileNotFoundError(f'{pickle_file_path} does not exist.')

# Step 3: Compress the model file
try:
    # Change the directory to 'model' and create the tar file with the pickle file at the root
    subprocess.run(
        ['tar', '-czvf', tar_file_path, '-C', os.path.dirname(pickle_file_path), os.path.basename(pickle_file_path)],
        check=True
    )
    print(f'Successfully compressed {pickle_file_path} to {tar_file_path}')
except subprocess.CalledProcessError as e:
    print(f'Error compressing file: {e}')



Deleted existing file: artifacts/xgb_model.tar.gz
Successfully compressed artifacts/xgb_model.pkl to artifacts/xgb_model.tar.gz


Step 2

In [18]:
s3_bucket = 'kevindouanla'
s3_bucket_path = f's3://{s3_bucket}'

# Step 4: Upload the compressed file to S3
try:
    s3_client = boto3.client('s3')
    # Upload the file to S3
    s3_client.upload_file(tar_file_path, s3_bucket, tar_file_path)
    print(f"File uploaded successfully to s3://{s3_bucket}/{tar_file_path}")
except Exception as e:
    print(f"Error uploading file: {e}")

# Step 5: Output the S3 location of the model data
model_data = f'{s3_bucket_path}/{tar_file_path}'
print(f"Model Data S3 Location: {model_data}")

File uploaded successfully to s3://kevindouanla/artifacts/xgb_model.tar.gz
Model Data S3 Location: s3://kevindouanla/artifacts/xgb_model.tar.gz


step 3

In [31]:
from sagemaker import get_execution_role
from sagemaker.model import Model
import sagemaker

In [42]:

role = "arn:aws:iam::077260319067:role/aws-role-sagemaker"

sagemaker_session = sagemaker.Session()

image_uri = "077260319067.dkr.ecr.eu-north-1.amazonaws.com/aws-model-ecr:latest"
entry_point = "inference.py"
source_dir = "src"

model = Model(
    image_uri=image_uri, # Provide the URI to your Pushed Docker image from AWS ECR
    model_data=model_data,
    role=role,
    sagemaker_session=sagemaker_session,
    entry_point='inference.py',
    source_dir='.',
    env={
        'SAGEMAKER_CONTAINER_LOG_LEVEL': '30',
        'SAGEMAKER_ENABLE_CLOUDWATCH_LOGGING': 'true'
    }
)

# Create and deploy the endpoint
try:
    predictor = model.deploy(
        instance_type="ml.m5.xlarge",
        initial_instance_count=1,
        endpoint_name="your-good-endpoint-name"
    )
except Exception as e:
    print(f"Failed to deploy endpoint: {e}")


Failed to deploy endpoint: An error occurred (ValidationException) when calling the CreateModel operation: Unsupported manifest media type application/vnd.oci.image.manifest.v1+json for image 077260319067.dkr.ecr.eu-north-1.amazonaws.com/aws-model-ecr:latest. Ensure that valid manifest media type is used for specified image.
